# Example 3: Grandma Portfolio Allocation

This example demostrates how to build a managable portfolio using the Grandma Portfolio Allocation.

This notebook contains a a lot of content. You may want to run through the codes first, before reading the mathematical details.

* Load Data
* Portfolio Allocation - the Basic Idea
* Build a Portfolio with SP500 ETF and Cash
    * Significance of Cash as Part of Portfolio
* Grandma Portfolio Allocation - Multiple Instruments
    * Correlation Weight
    * Portfolio Allocation - the Details


In [1]:
from grandma_stock_valuation import FileLogger, loadPacakgeData
from grandma_stock_valuation import batchValuation, addCashPortfolio, getCorrelationWeight, allocatePortfolio

# Refer to example_0_FileLogger.ipynb for details of the FileLogger.
logger = FileLogger()
logPrint = logger.logPandas

### Load Data

For this example, I will use the stored package data.

To query data from Yahoo, please refer to *example_1_yahoo_data_loader.ipynb*.

In [2]:
d_instrument_data, d_instrument = loadPacakgeData(verbose=2)

d_instrument

VPL data contains 4273 rows, 4273 dates from 2005-03-10 to 2022-02-28.
IVV data contains 5479 rows, 5479 dates from 2000-05-19 to 2022-02-28.
EEMA data contains 2530 rows, 2530 dates from 2012-02-09 to 2022-02-28.
IEV data contains 5431 rows, 5431 dates from 2000-07-28 to 2022-02-28.


{'IVV': 'SP500',
 'VPL': 'Developed Asia-Pacific',
 'IEV': 'Europe',
 'EEMA': 'Emerging Asia'}

In [3]:
logPrint("Keys of d_instrument_data:", str(d_instrument_data.keys()))

logPrint("IVV (SP500 ETF):", d_instrument_data['IVV'].head())

2022-03-03 21:48:05,223 INFO Keys of d_instrument_data: dict_keys(['VPL', 'IVV', 'EEMA', 'IEV'])
2022-03-03 21:48:05,250 INFO IVV (SP500 ETF): 
        date       open       high        low      close  close_adj   volume
0 2000-05-19  142.65625  142.65625  140.25000  140.68750  94.121216   775500
1 2000-05-22  140.59375  140.59375  136.81250  139.81250  93.535789  1850600
2 2000-05-23  140.21875  140.21875  137.68750  137.68750  92.114151   373900
3 2000-05-24  137.75000  140.06250  136.65625  139.75000  93.494003   400300
4 2000-05-25  140.03125  140.93750  137.87500  138.46875  92.636810    69600



### Portfolio Allocation - the Basic Idea

*Should I buy or sell?*

Is this question daunting you all the time?

Well, this is an important question. But let's take a step back - how about thinking investment as **portfolio management**?

*What is it about?*

**Portfolio management** is both familiar and new to retail investors.
* On one hand, many people have heard about it, and some even have knowledge about certain techniques.
* On the other hand, most people don't know how to put it into practice.

Let's go straight into the points:
* Is portfolio management about **decipline**? - not really - rather a way of thinking.
* **How frequent** do I need to adjust my portfolio? - about monthly - not too frequent, but don't leave your investment unattended.
* **What instruments** shall I put in my portfolio? - `Grandma` suggests broad ETF.
* **How many instruments** shall I hold? - I suggest fewer than 10 - `Grandma` will help you with it.
* **How much money** shall I allocate to each instrument? - higher % to under-valued instruments - `Grandma` will give you the number.
* **How much CASH** shall I place aside? - `Grandma` will answer it.

### Build a Portfolio with SP500 ETF and Cash

Let's start with a simple portfolio consists of the **SP500 ETF (IVV)** and **cash**.

*What? Can I call this a portfolio?* - Of course!

Cell below get valuation of the SP500 ETF using Grandma Stock Valuation model - refer to *example_2_grandma_valuation.ipynb* if you are not familiar with it yet.

In [4]:
d_SP500 = {ticker:daily_price for ticker, daily_price in d_instrument_data.items() if ticker=='IVV'}

df_SP500_valuation, _ = batchValuation(d_SP500, save_result=False)

df_SP500_valuation

,ticker,r2_train,train_years,annualized_return,current_price,fair_price,over_value_range,over_value_years
0,IVV,0.971054,10.00274,0.134314,438.720001,411.907796,0.065093,0.48463


We can use the `addCashPortfolio()` function to add cash into this valuation table. `addCashPortfolio()` takes the following arguments:
* `df_valuation_metrics` (pandas.DataFrame): a dataframe with the valuation metrics of an existing portfolio.<br>It is usually the output of `batchValuation()` function.
* `id_col` (str): column in `df_valuation_metrics` as the identifier of the instruments in the portfolio.<br>Default to "ticker".
* `cash_name` (str): a name, default to "cash", to be appended under the `id_col`.
* `value_col` (str): column in `df_valuation_metrics` with the valuations of each instruments in the portfolio.<br>Default to `over_value_years`.
* `cash_value` (float): Valuation of cash. I will just use 0 (neither over-valued nor under-valued).<br>You may want to take inflation or other factor into consideration - up to your decision.



In [5]:
df_SP500_and_cash = addCashPortfolio(df_SP500_valuation, cash_value=0.0)

df_SP500_and_cash

,ticker,r2_train,train_years,annualized_return,current_price,fair_price,over_value_range,over_value_years
0,IVV,0.971054,10.00274,0.134314,438.720001,411.907796,0.065093,0.48463
1,cash,NaN,NaN,NaN,NaN,NaN,NaN,0.00000


`allocatePortfolio()` will tell you how much % to allocate to each instrument:

In [6]:
df_SP500_and_cash['allocation'] = allocatePortfolio(df_SP500_and_cash['over_value_years'], with_cash=True)

df_SP500_and_cash

,ticker,r2_train,train_years,annualized_return,current_price,fair_price,over_value_range,over_value_years,allocation
0,IVV,0.971054,10.00274,0.134314,438.720001,411.907796,0.065093,0.48463,0.381159
1,cash,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.618841


**OK!** - before going into how `allocatePortfolio()` works in detail, let's take a look at this result.
* Because SP500 was over-valued (by 0.48 years), it was allocated with 38% of your total portfolio (or budget).
* Cash got higher allocation than SP500, because cash was less over-valued than SP500.

If SP500 further grows, the model will suggest less allocation to SP500 and more to cash - "sell" in practice.

If SP500 drops and even becomes under-valued, the model will suggest more allocation to SP500 and less to cash - "buy" in practice.

See, you can **"Shall I buy or sell?"** now! Simple isn't it?

<br>

#### Significance of Cash as Part of Portfolio

I want to explain this important point first, before we move further.

It is optional to have cash as part of a portfolio - you can put 100% of your budget into some stocks - and this is how most mutual funds work.
* But you are not a mutual fund - you don't earn management fee!
* Please remeber, **stocks are stocks, not cash**. Stocks can be useless when you urgently need money to support your business amid the Covid-19 situation. You only earn money when you sell the stocks and put cash into your pocket.
* A complete investment strategy needs to include a **harvest mechanism** for you to secure your earning in cash.

The Grandma Portfolio Allocation provides you the option to include cash as part of your portfolio. As you can see from the previous SP500 example, this mechanism actual buys stocks when the valuation(s) is low, and sells - **harvests your earnings** - when the valuation(s) is high.

As a result, I encourage you to always include cash as part of your portfolio, and let it managed by the `Grandma`.

### Grandma Portfolio Allocation - Multiple Instruments

Now let's built a portfolio with a close-to-global coverage:
* IVV: SP500 ETF
* IEV: Europe ETF
* VPL: Developed Asia-Pacific ETF
* EEMA: Emerging Asia ETF

The valuation (as of 2022-02-28) can be computated as below:

In [7]:
df_valuation, _ = batchValuation(d_instrument_data, save_result=False)
df_valuation = addCashPortfolio(df_valuation, cash_value=0.0)

df_valuation

,ticker,r2_train,train_years,annualized_return,current_price,fair_price,over_value_range,over_value_years
0,VPL,0.890218,10.00274,0.067348,74.18,77.196246,-0.039072,-0.263147
1,IVV,0.971054,10.00274,0.134314,438.720001,411.907796,0.065093,0.484630
2,EEMA,0.847897,10.00274,0.065518,77.550003,79.470586,-0.024167,-0.158339
3,IEV,0.780975,10.00274,0.052644,50.130001,49.242054,0.018032,0.342530
4,cash,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


#### Correlation Weight

The final portfolio allocation is determined by two factors - a "weight" and the valuations.

* The "weight" can be selected arbitrarily by your preference - for example, you may have a strong personal interest to invest more in SP500.
* Or, there can no weight - allocation purely based on valuations.
* Or, a common quantitative practice - the **correlation weight**.

Probably you are already quite familar with correlation - yes, it is about **diversification**:
* Highly correlated instruments won't reduce the overall risk, because their price trend to go up and down together.
* In general you want to build a portfolio with instruments not so correlated with each other.

But how to **implement this concept in a quantitative manner**? Let me give you an example:

* Say, we have two instruments, `A` and `B`, and they are not so correlated:
    * `cor(A, B) = cor(B, A) = 0.4`
* Without additional information, let's equally divide our portfolio as `A = 50%, B = 50%`.
* Now, there is a new instrument `C` which is totally correlated to `B`
    * `cor(B, C) = cor(C, B) = 1`
    * `cor(A, C) = cor(C, A) = 0.4`
* Without additional information, the portfolio should become `A = 50%, B = 25%, C = 25%`
    * Please think about this logic carefully, because the math is build on top of this.

Below is the math to make it happen:
* For instrument `A`:
    * `cor(A, B) = 0.4, cor(A, C) = 0.4`
    * `Score(A) = 1 - cor(A, B) + 1 - cor(A, C) = 1.2`
* For instrument `B`:
    * `cor(B, A) = 0.4, cor(B, C) = 1`
    * `Score(B) = 1 - cor(B, A) + 1 - cor(B, A) = 0.6`
* For instrument `C`:
    * `cor(C, A) = 0.4, cor(C, B) = 1`
    * `Score(C) = 1 - cor(C, A) + 1 - cor(C, B) = 0.6`
* Now for the correlation weights:
    * `A = Score(A) / SUM(Score A, B, C) = 1.2 / (1.2 + 0.6 + 0.6) = 50%`
    * `B = Score(B) / SUM(Score A, B, C) = 0.6 / (1.2 + 0.6 + 0.6) = 25%`
    * `C = Score(C) / SUM(Score A, B, C) = 0.6 / (1.2 + 0.6 + 0.6) = 25%`

This math can be easily scaled up to more than 3 instruments. The `getCorrelationWeight()` function is provided to you, which takes the following arguments:
* `d_instrument_prices` (dict): Dictionary with the daily price data of each instrument. Refer to `d_instrument_data` loaded at the beginning of the notebook.
* `price_col` (str): The column name to in the data to indicate daily price. Default to `close_adj`.
* `recent_months` (str): Number of recent months, before `date_end`, to exclude from correlation computation. Default to 0.
* `train_years` (str): Years of historical data, after excluding `recent_months`, for correlation computation. Default to 10 years.
* `date_end` (date): Data after this date will not be used. Default to None.
* `with_cash` (bool): If True, a fixed weight will be allocate to cash before computing the correlation. I will expalin this later.
* `cash_name` (str): Name of cash to be presented as key in the output dictionary. Default to `cash`.
* `verbose` (int): 2 to print detailed information; 1 to print high-level information; 0 to suppress print.
* `printfunc` (function): function to output messages. Default to `print`.

Let's first see the result of `with_cash=False`:

In [8]:
getCorrelationWeight(d_instrument_data, with_cash=False, verbose=2, printfunc=logPrint)

2022-03-03 22:01:30,721 INFO VPL: Selected 2516 rows over 2516 dates from 2012-03-01 to 2022-02-28.
2022-03-03 22:01:30,732 INFO IVV: Selected 2516 rows over 2516 dates from 2012-03-01 to 2022-02-28.
2022-03-03 22:01:30,742 INFO EEMA: Selected 2516 rows over 2516 dates from 2012-03-01 to 2022-02-28.
2022-03-03 22:01:30,753 INFO IEV: Selected 2516 rows over 2516 dates from 2012-03-01 to 2022-02-28.


{'VPL': 0.1666199308736074,
 'IVV': 0.2752411752938485,
 'EEMA': 0.2481453204446346,
 'IEV': 0.30999357338790945}

The total weight sums up to 100%:
* IEV (Europe) gets the highest weight, because it is least correlated with other ETFs (SP500 / Asia).
* VPL (Developed Asia-Pacific) gets the lowest weight, because it is more correlated with other ETFs.

Now let's see the result of `with_cash=True`:

In [9]:
getCorrelationWeight(d_instrument_data, with_cash=True, verbose=0, printfunc=logPrint)

{'VPL': 0.13329594469888595,
 'IVV': 0.22019294023507885,
 'EEMA': 0.19851625635570772,
 'IEV': 0.2479948587103276,
 'cash': 0.2}

Let me explain the result above:
* Cash always has little correlation with stocks.
* If using the same formulation as stocks, cash will get a huge correlation weight.
* To cope with this, we need to handle cash separately.

In this case, because there are totally 5 instruments (including cash):
* Firstly, cash was allocated with fixed 100% / 5 = 20%
* Then the remaining 80% was allocated to the four ETFs based on correlation.




#### Portfolio Allocation - the Details

We have talked about the first factor of portfolio allocation - correlation weight. Now we will deep dive into the valuation part.

